# Deploy chatbot to Gradio

In [3]:
# 📦 Imports
import gradio as gr
import whisper
import uuid
import os
from gtts import gTTS
import utils  # Import the whole module so you can reload it
import importlib

importlib.reload(utils)

# Then you can safely import the individual functions again
from utils import init_qa_chain, get_agent


# ✅ Load QA agent
qa_chain = init_qa_chain()
agent = get_agent(qa_chain)

# ✅ Load Whisper model once
model = whisper.load_model("base")

# 🔧 Utility: Transcribe audio file using Whisper
def transcribe_audio_file(audio_file):
    print("📝 Transcribing audio...")
    result = model.transcribe(audio_file.name)
    text = result["text"].strip()
    print(f"🗣️ Transcribed text: {text}")
    return text

# 🔊 Utility: Generate TTS file using gTTS
def generate_tts(text):
    filename = f"{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    return filename

def shorten_text(text, max_ratio=0.5):
    prompt = f"Please shorten the following text to about {int(max_ratio*100)}% of its length, keeping the key information:\n\n{text}"
    shortened = agent.run(prompt)
    return shortened

# 🧠 Ask the agent
def answer_with_sources(question):
    prompt = f"Use the YouTubeTranscriptQA tool to answer this question: {question}"
    full_answer = agent.run(prompt)
    short_answer = shorten_text(full_answer)
    return short_answer

# 🧩 Gradio logic: handles text or audio
def gradio_qa(text_input, audio_input):
    if text_input and text_input.strip():
        question = text_input.strip()
        print("✅ Using text input")
    elif audio_input is not None:
        question = transcribe_audio_file(audio_input)
        print("✅ Using audio input")
    else:
        return "Please enter a question or upload audio.", "", None

    answer = answer_with_sources(question)
    audio_path = generate_tts(answer)
    return question, answer, audio_path

# 🎛️ Gradio interface
iface = gr.Interface(
    fn=gradio_qa,
    inputs=[
        gr.Textbox(label="Enter your question (text)", lines=2, placeholder="Type your question here..."),
        gr.Audio(type="filepath", label="Or upload your question as audio")
    ],
    outputs=[
        gr.Textbox(label="Processed Question"),
        gr.Textbox(label="Answer"),
        gr.Audio(label="Answer Audio", type="filepath")
    ],
    title="YouTube Transcript QA (Text & Voice Input)",
    description="Ask questions via text or audio and get answers based on YouTube transcripts, including spoken output.",
    allow_flagging="never"
)

# 🟢 Launch interface
if __name__ == "__main__":
    iface.launch(share=True)


/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:21: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=keys["OPENAI_API_KEY"])
/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=key

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://eaed4364cc5d00a790.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Using text input


> Entering new AgentExecutor chain...


/var/folders/hq/l56ghxv518j9wg6pgqkbbvd80000gn/T/ipykernel_32937/572740301.py:46: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  full_answer = agent.run(prompt)


Thought: Do I need to use a tool? Yes
Action: YouTubeTranscriptQA
Action Input: who is bill nye?

/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:65: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": input_text})



Observation: Bill Nye is a popular American science communicator, television presenter, and mechanical engineer. He is best known as the host of the PBS children's science show "Bill Nye the Science Guy" (1993–1998), which aimed to teach a specific topic in science to a preteen audience.

Sources:
40 - Neil deGrasse Tyson and Bill Nye Catch Up.en.txt
Thought:Do I need to use a tool? No
AI: Bill Nye is a popular American science communicator, television presenter, and mechanical engineer. He is best known as the host of the PBS children's science show "Bill Nye the Science Guy" (1993–1998), which aimed to teach a specific topic in science to a preteen audience.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Bill Nye, an American science communicator and TV presenter, is renowned for hosting the PBS children's show "Bill Nye the Science Guy" (1993–1998), educating preteens on various science topics.

> Finished chain.
